### Set directory and session info

In [1]:
date = '210126'
fov = 'FOV1'
badTrials = []
imgPath = '/Volumes/genie/Jihong_GENIE_stuff/NMJ/uf-GCaMP/'
# imgPath = '/Users/zhengj10/Desktop/ufGCaMP/'+ date + '/' + larvaID + '/' + fov
shutterCloseFrames = 0


### Import modules

In [2]:
import numpy as np
import os
import shutil
from registration import CrossCorr
from skimage.io import imsave
from glob import glob
import tifffile as tff
import matplotlib.pyplot as plt


### Load data

In [3]:
imgFiles = glob(imgPath + '/TSeries' + '*.tif')
imgFiles = sorted(imgFiles)
# for imgFile in imgFiles:
#     print (imgFile)
print ('The number of trials is: ', len(imgFiles))
os.path.exists(imgPath)

The number of trials is:  0


True

### Registration

In [5]:
algorithm = CrossCorr()
for i in range(1,5):
    
    reference = tff.TiffFile(imgPath + date + '/' + 'Larva_' + str(i) + '/' + fov + '/summary/meanOfImg_080Hz.tif')
    reference = reference.asarray()
    reference = np.mean(reference, axis=0)
    print (reference.shape)
    sumPath = imgPath + date + '/' + 'Larva_' + str(i) + '/' + fov + '/tdtomato/'
    imgFiles = glob(sumPath + '*.tif')
    
    data = tff.TiffFile(imgFiles[0])

    print ('currently registering ' + str(i))
    data = data.asarray()
    print(data.shape)
    model = algorithm.fit(data, reference)
    registered = model.transform(data)
    registered = registered.toarray()
    mean = np.mean(registered,axis=0)
    imsave(sumPath + '/mean.tif',mean.astype('float32'), plugin='tifffile', photometric='minisblack')
    print('done')


(128, 128)
currently registering 1
(1800, 128, 128)
done
(128, 128)
currently registering 2
(1800, 128, 128)
done
(128, 128)
currently registering 3
(1800, 128, 128)
done
(128, 128)
currently registering 4
(1800, 128, 128)
done


In [ ]:
fig = plt.figure()
plt.imshow(reference, cmap = 'gray')
# fig.savefig('/Users/zhengj10/Desktop/jGCaMP7f_NMJ.png')
plt.show()



In [ ]:
# xStart = 150
# xEnd = 400
# yStart = 200
# yEnd = 350

xStart = 0
xEnd = 127
yStart = 40
yEnd = 100

vmin = reference.min() 
vmax = reference.max()

from matplotlib import gridspec
fig = plt.figure(figsize=(12, 6)) 
gs = gridspec.GridSpec(1, 2, width_ratios=[1, (xEnd-xStart)/512.0])
ax0 = plt.subplot(gs[0])
ax0.imshow(reference, cmap = 'gray', vmin = vmin, vmax = vmax)
ax0.plot([yStart,yEnd,yEnd,yStart,yStart],[xEnd,xEnd,xStart,xStart,xEnd], 'y')
ax1 = plt.subplot(gs[1])
ax1.imshow(reference[xStart:xEnd, yStart:yEnd], cmap = 'gray', vmin = vmin, vmax = vmax)

# plt.tight_layout()



# plt.figure(figsize=(10,10))
# plt.imshow(reference, cmap = 'gray', vmin = vmin, vmax = vmax)
# plt.plot([yStart,yEnd,yEnd,yStart,yStart],[xEnd,xEnd,xStart,xStart,xEnd],'y')
# plt.figure(figsize=(10,10))
# plt.imshow(reference[xStart:xEnd, yStart:yEnd], cmap = 'gray', vmin = vmin, vmax = vmax)
plt.show()


In [ ]:
trialProtoc = { 
                5: '001Hz',
               10: '005Hz',
               15: '010Hz',
               20: '020Hz',
               25: '040Hz',
               30: '080Hz',
               35: '160Hz',
               40: '240Hz',
               45: '320Hz'
              }

sumPath = os.path.join(imgPath, 'summary')
if not os.path.exists(sumPath):
    os.makedirs(sumPath)

algorithm = CrossCorr()

meanOfImg = []
# for trialNum in range(len(imgFiles)):
for trialNum in range(35,40):
    if trialNum+1 in badTrials:
        continue
    data = tff.TiffFile(imgFiles[trialNum])
    print ('currently registering ' + str(trialNum+1))
    data = data.asarray()
    data = data[shutterCloseFrames:, :, :]
    model = algorithm.fit(data[:, xStart:xEnd, yStart:yEnd], reference[xStart:xEnd, yStart:yEnd])
    registered = model.transform(data)
    registered = registered.toarray()
    print (os.path.splitext(os.path.basename(imgFiles[trialNum]))[0] + '.tif Registered')
    meanOfImg.append(registered)
    if (trialNum+1)%5 == 0:

        meanOfImg = np.mean(meanOfImg,axis=0)
        print (meanOfImg.shape)
        imsave(sumPath + '/meanOfImg_' + trialProtoc[trialNum+1] + '.tif', meanOfImg.astype('float32'), plugin='tifffile', photometric='minisblack')
#         imsave(sumPath + '/meanOfImg_160Hz.tif', meanOfImg.astype('float32'), plugin='tifffile', photometric='minisblack')
        meanOfImg = []
        print  (trialProtoc[trialNum+1] + ' trials average image saved')
print ('Trials averaging completed')
